In [1]:
%%capture
!pip install librosa
!pip install torch

In [2]:
import os
import pandas as pd
import s3fs
import zipfile
import torch
from torch.utils.data import Dataset
import numpy as np
import matplotlib.pyplot as plt
import librosa
from scipy import signal
from tqdm import tqdm
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from torch.utils.data import DataLoader
from sklearn.ensemble import RandomForestRegressor
from Misc import Misc
from CustomDataset import CustomDataset
from EuclideanDistanceLoss import EuclideanDistanceLoss

In [3]:
DATASET_PATH = "../data/LivingRoom_preprocessed_hack"
# Dimensions of the living room
X_MIN = -4000
X_MAX = 500
Y_MIN = -4000
Y_MAX = 2000
MISC = Misc(start_time=0, end_time=50000, sr=44100, target_sr=16000)
EUCLIDEAN_LOSS = EuclideanDistanceLoss()

## Read Data

In [ ]:
centroids_h1 = np.load(DATASET_PATH + "/Human1/centroid.npy")
centroids_h2 = np.load(DATASET_PATH + "/Human2/centroid.npy")
deconvoled_trim_h1 = np.load(DATASET_PATH + "/Human1/deconvoled_trim.npy")
deconvoled_trim_h2 = np.load(DATASET_PATH + "/Human2/deconvoled_trim.npy")

In [ ]:
print("Deconvoled Trim Human 1 Shape: ", deconvoled_trim_h1.shape)
print("Deconvoled Trim Human 2 Shape: ", deconvoled_trim_h2.shape)
print("Centroids Huma 1 Shape: ", centroids_h1.shape)
print("Centroids Human 2 Shape: ", centroids_h2.shape)

## Preprocessing

In [ ]:
preprocessed_data_mfcc_h1, preprocessed_data_rms_h1, preprocessed_data_zcr_h1 = MISC.preprocess_knn(deconvoled_trim=deconvoled_trim_h1)
preprocessed_data_mfcc_h2, preprocessed_data_rms_h2, preprocessed_data_zcr_h2 = MISC.preprocess_knn(deconvoled_trim=deconvoled_trim_h2)

In [ ]:
print("Preprocessed RMS Human 1 data: ", preprocessed_data_rms_h1.shape)
print("Preprocessed Zero-Crossing Rate Human 1 data: ", preprocessed_data_zcr_h1.shape)
print("Preprocessed MFCC Human 1 data: ", preprocessed_data_mfcc_h1.shape)
print("Preprocessed RMS Human 2 data: ", preprocessed_data_rms_h2.shape)
print("Preprocessed Zero-Crossing Rate Human 2 data: ", preprocessed_data_zcr_h2.shape)
print("Preprocessed MFCC Human 2 data: ", preprocessed_data_mfcc_h2.shape)

In [ ]:
# Plot audio features for each instance and channel
for instance_index in range(preprocessed_data_mfcc_h2.shape[0]):
    for channel_index in range(preprocessed_data_mfcc_h2.shape[1]):
        mfcc_features = preprocessed_data_mfcc_h2[instance_index, channel_index]
        MISC.plot_audio_features(instance_index=instance_index, chan_index=channel_index, mfcc=mfcc_features)
    break

# Plot audio features for each instance and channel
for instance_index in range(preprocessed_data_zcr_h2.shape[0]):
    for channel_index in range(preprocessed_data_zcr_h2.shape[1]):
        zero_crossing_rate = preprocessed_data_zcr_h2[instance_index, channel_index] 
        MISC.plot_audio_features(instance_index=instance_index, chan_index=channel_index, zcr=zero_crossing_rate)
    break

## Dataset

In [ ]:
dataset_h1 = CustomDataset(preprocessed_data_rms=preprocessed_data_rms_h1, coordinates=centroids_h1)
dataset_h2 = CustomDataset(preprocessed_data_rms=preprocessed_data_rms_h2, coordinates=centroids_h2)

# Access a sample from the dataset
features_list, coordinates = dataset_h1[1]
print("Features List Length:", len(features_list))
print("Features Shape (Microphone 1):", features_list[0])
print("Features Shape (Microphone 2):", features_list[1])
print("Features Shape (Microphone 3):", features_list[2])
print("Features Shape (Microphone 4):", features_list[3])
print("Coordinates:", coordinates)

## Random Forest Model Training

In [ ]:
criterion = EUCLIDEAN_LOSS

def custom_scoring(self, estimator, X, y):
        pred_coords = estimator.predict(X)
        ed = np.mean([criterion(torch.tensor(p), torch.tensor(t)) for p, t in zip(pred_coords, y)])
        return -ed

In [ ]:
rf_regressor = RandomForestRegressor(random_state=42)

param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [5, 10, 15]
}

grid_search = GridSearchCV(rf_regressor, param_grid, cv=5, scoring=custom_scoring) 
grid_search.fit(dataset_h1.preprocessed_data_rms, dataset_h1.coordinates)

print("Best Parameters:", grid_search.best_params_)
print("GridSearchCV Best Scoring:", -grid_search.best_score_)  # Negate for readability

In [ ]:
rf_model = RandomForestRegressor(random_state=42, n_estimators=grid_search.best_params_['n_estimators'], max_depth=grid_search.best_params_['max_depth'])
rf_model.fit(dataset_h1.preprocessed_data_rms, dataset_h1.coordinates)

predicted_coordinates = rf_model.predict(dataset_h2.preprocessed_data_rms)

mse = mean_squared_error(centroids_h2, predicted_coordinates)
print("Mean Squared Error:", mse)

mae = mean_absolute_error(centroids_h2, predicted_coordinates)
print("Mean Absolute Error:", mae)

# Calculate localization errors for each prediction
errors = []
for pred_coords, true_coords in zip(predicted_coordinates, dataset_h2.coordinates):
    error = euclidean_distance(pred_coords, true_coords)
    errors.append(error)

errors = np.array(errors)
mean_error = np.mean(errors)
stdev_error = np.std(errors)

print("Localization Error: {:.2f} ({:.2f})".format(mean_error, stdev_error))
print("Score: ", rf_model.score(dataset_h2.preprocessed_data_rms, dataset_h2.coordinates))

In [ ]:
# Concatenate all true and predicted coordinates from all data points
all_true_coords = dataset_h2.coordinates.reshape(-1, 2)
all_pred_coords = predicted_coordinates.reshape(-1, 2)

# Plot all true and predicted coordinates on a single graph
plt.figure(figsize=(8, 6))
plt.scatter(all_true_coords[:, 0], all_true_coords[:, 1], color='blue', label='True Coordinates')
#plt.scatter(all_pred_coords[:, 0], all_pred_coords[:, 1], color='red', label='Predicted Coordinates')

plt.xlabel('X')
plt.ylabel('Y')
plt.title('True vs Predicted Coordinates')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Concatenate all true and predicted coordinates from all data points
all_true_coords = dataset_h2.coordinates.reshape(-1, 2)
all_pred_coords = predicted_coordinates.reshape(-1, 2)

# Plot all true and predicted coordinates on a single graph
plt.figure(figsize=(8, 6))
#plt.scatter(all_true_coords[:, 0], all_true_coords[:, 1], color='blue', label='True Coordinates')
plt.scatter(all_pred_coords[:, 0], all_pred_coords[:, 1], color='red', label='Predicted Coordinates')

plt.xlabel('X')
plt.ylabel('Y')
plt.title('True vs Predicted Coordinates')
plt.legend()
plt.grid(True)
plt.show()

## Zero-Crossing Rate

In [ ]:
dataset_h1 = CustomDataset(preprocessed_data_zcr=preprocessed_data_zcr_h1, _type="zcr", coordinates=centroids_h1)
dataset_h2 = CustomDataset(preprocessed_data_zcr=preprocessed_data_zcr_h2, _type="zcr", coordinates=centroids_h2)

rms_features = np.sqrt(np.mean(dataset_h1.preprocessed_data_zcr**2, axis=-1))
X_train = rms_features.reshape(1000, 4)
y_train = dataset_h1.coordinates
rms_features = np.sqrt(np.mean(dataset_h2.preprocessed_data_zcr**2, axis=-1))
X_test = rms_features.reshape(104, 4)
y_test = dataset_h2.coordinates

rf_regressor = RandomForestRegressor(random_state=42)

param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [5, 10, 15]
}

grid_search = GridSearchCV(rf_regressor, param_grid, cv=5, scoring=custom_scoring) 
grid_search.fit(X_train, y_train)

rf_model = RandomForestRegressor(random_state=42, n_estimators=500, max_depth=30)
rf_model.fit(X_train, y_train)

predicted_coords = rf_model.predict(X_test)

mse = mean_squared_error(y_test, predicted_coords)
print("Mean Squared Error:", mse)

mae = mean_absolute_error(y_test, predicted_coords)
print("Mean Absolute Error:", mae)

# Calculate localization errors for each prediction
errors = []
for pred_coords, true_coords in zip(predicted_coords, y_test):
    error = euclidean_distance(pred_coords, true_coords)
    errors.append(error)

errors = np.array(errors)
mean_error = np.mean(errors)
stdev_error = np.std(errors)

print("Localization Error: {:.2f} ({:.2f})".format(mean_error, stdev_error))
print("Score: ", rf_model.score(X_test, y_test))